## GraphSAGE: Inductive Representation Learning on Large Graphs

**GraphSAGE**是一个用于大型图上归纳表示学习的框架。

**GraphSAGE**用于为节点生成低维向量表示, 对于具有丰富节点属性信息的图尤其有用。

### Import packages

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init